In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [8]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [10]:
num_classes = 10

In [11]:
x_train = x_train.reshape(-1, 784).astype('float32')
x_test = x_test.reshape(-1, 784).astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

60000 train samples
10000 test samples


In [19]:
model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('sigmoid'),
    Dense(10),
    Activation('softmax'),
])

In [22]:
model.reset_states() 
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model.reset_states()
# Train the model, iterating on the data in batches of 32 samples
training_history = model.fit(x_train, y_train, epochs=4, batch_size=64)

Epoch 1/4
60000/60000 [==============================] - 3s 45us/step - loss: 0.0312 - acc: 0.9901
Epoch 2/4
60000/60000 [==============================] - 3s 44us/step - loss: 0.0285 - acc: 0.9910
Epoch 3/4
60000/60000 [==============================] - 3s 44us/step - loss: 0.0262 - acc: 0.9917
Epoch 4/4
60000/60000 [==============================] - 3s 43us/step - loss: 0.0245 - acc: 0.9923


In [24]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 0s 47us/step


[0.03291190845929086, 0.9893200006484986]

In [66]:
from keras.utils import plot_model
plot_model(model)